In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bile_Duct_Cancer"
cohort = "GSE107754"

# Input paths
in_trait_dir = "../../input/GEO/Bile_Duct_Cancer"
in_cohort_dir = "../../input/GEO/Bile_Duct_Cancer/GSE107754"

# Output paths
out_data_file = "../../output/preprocess/Bile_Duct_Cancer/GSE107754.csv"
out_gene_data_file = "../../output/preprocess/Bile_Duct_Cancer/gene_data/GSE107754.csv"
out_clinical_data_file = "../../output/preprocess/Bile_Duct_Cancer/clinical_data/GSE107754.csv"
json_path = "../../output/preprocess/Bile_Duct_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A novel genomic signature predicting FDG uptake in diverse metastatic tumors"
!Series_summary	"Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake."
!Series_summary	"Methods: A balanced training set (n=71) of metastatic tumors including some of the most frequent histologies, with matched PET/CT quantification measurements and whole human genome gene expression microarrays, was used to build the signature. Selection of microarray features was carried out exclusively on the basis of their strong association with FDG uptake (as measured by SUVmean35) by means of univariate linear regression. A thorough bioinformatics study of these genes was performed and multivariable models were built by fitting several state of the art regression techniques to the training set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze gene expression data availability
# From the background information, this dataset appears to contain whole human genome gene expression microarrays,
# which suggests gene expression data is available
is_gene_available = True

# 2.1 Identify available data rows
# For trait data: looking at row 2 which contains tissue information including "Bile duct cancer"
trait_row = 2  # This contains tissue types including bile duct cancer

# Age is not available in the sample characteristics dictionary
age_row = None

# Gender is available in row 0
gender_row = 0

# 2.2 Define conversion functions
def convert_trait(value):
    """Convert tissue values to binary for bile duct cancer."""
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates bile duct cancer
    # The exact match in the data is 'tissue: Bile duct cancer'
    if 'bile duct cancer' in value.lower():
        return 1
    else:
        return 0

def convert_age(value):
    """
    Since age data is not available, this function is a placeholder.
    In real use, it would convert age values to a continuous format.
    """
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to lowercase for case-insensitive comparison
    value_lower = value.lower()
    
    if 'female' in value_lower:
        return 0
    elif 'male' in value_lower:
        return 1
    else:
        return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If trait data is available, extract clinical features
if trait_row is not None:
    # Create directory for clinical data if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Select clinical features from clinical_data DataFrame
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assumed to be pre-loaded from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(clinical_features)
    print(f"Preview of clinical features: {preview}")
    
    # Save the clinical features to the output file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {'GSM2878070': [0.0, 1.0], 'GSM2878071': [0.0, 0.0], 'GSM2878072': [0.0, 1.0], 'GSM2878073': [0.0, 1.0], 'GSM2878074': [0.0, 0.0], 'GSM2878075': [0.0, 1.0], 'GSM2878076': [0.0, 0.0], 'GSM2878077': [0.0, 0.0], 'GSM2878078': [0.0, 0.0], 'GSM2878079': [0.0, 0.0], 'GSM2878080': [0.0, 0.0], 'GSM2878081': [0.0, 0.0], 'GSM2878082': [0.0, 0.0], 'GSM2891194': [0.0, 1.0], 'GSM2891195': [0.0, 1.0], 'GSM2891196': [0.0, 0.0], 'GSM2891197': [0.0, 0.0], 'GSM2891198': [0.0, 0.0], 'GSM2891199': [0.0, 0.0], 'GSM2891200': [0.0, 0.0], 'GSM2891201': [0.0, 1.0], 'GSM2891202': [0.0, 1.0], 'GSM2891203': [0.0, 0.0], 'GSM2891204': [0.0, 0.0], 'GSM2891205': [0.0, 1.0], 'GSM2891206': [0.0, 0.0], 'GSM2891207': [0.0, 0.0], 'GSM2891208': [0.0, 1.0], 'GSM2891209': [0.0, 1.0], 'GSM2891210': [0.0, 0.0], 'GSM2891211': [0.0, 0.0], 'GSM2891212': [0.0, 0.0], 'GSM2891213': [0.0, 0.0], 'GSM2891214': [0.0, 0.0], 'GSM2891215': [0.0, 1.0], 'GSM2891216': [0.0, 0.0], 'GSM2891217': [0.0, 1.0], 'GSM289

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see that they start with "A_23_P" followed by numbers
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# Instead, these appear to be Agilent microarray probe IDs which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe contain the probe IDs and gene symbols
probe_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

# 2. Get the gene mapping dataframe using the two identified columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)

# 3. Convert probe-level measurements to gene expression data
# This applies the gene mapping and handles the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the resulting gene expression data
print("Gene expression data preview (after mapping):")
print(gene_data.shape)
print(gene_data.index[:20])


Gene expression data preview (after mapping):
(18488, 84)
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS', 'AADAC', 'AADACL2', 'AADAT', 'AAGAB', 'AAK1', 'AAMP',
       'AANAT', 'AARS', 'AARS2', 'AARSD1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. We need to first load or recreate the selected_clinical_df
try:
    # Try to load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    # Fix index to ensure proper row names
    selected_clinical_df.index = [trait, 'Gender']
    print("Loaded clinical data from file.")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data sample:")
    print(selected_clinical_df.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    print("Recreating clinical features from raw data...")
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

# Link the clinical and genetic data
print(f"Clinical data shape before linking: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to process gene expression data appropriately."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = "Dataset contains gene expression data from metastatic tumors including bile duct cancer samples with matched FDG uptake measurements."
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=linked_data, 
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 18488
Normalized gene count: 18247


Normalized gene data saved to ../../output/preprocess/Bile_Duct_Cancer/gene_data/GSE107754.csv
Loaded clinical data from file.
Clinical data shape: (2, 84)
Clinical data sample:
                  GSM2878070  GSM2878071  GSM2878072  GSM2878073  GSM2878074  \
Bile_Duct_Cancer         0.0         0.0         0.0         0.0         0.0   
Gender                   1.0         0.0         1.0         1.0         0.0   

                  GSM2878075  GSM2878076  GSM2878077  GSM2878078  GSM2878079  \
Bile_Duct_Cancer         0.0         0.0         0.0         0.0         0.0   
Gender                   1.0         0.0         0.0         0.0         0.0   

                  ...  GSM2891255  GSM2891256  GSM2891257  GSM2891258  \
Bile_Duct_Cancer  ...         0.0         1.0         0.0         0.0   
Gender            ...         1.0         1.0         1.0         1.0   

                  GSM2891259  GSM2891260  GSM2891261  GSM2891262  GSM2891263  \
Bile_Duct_Cancer         0.0         1.0

Linked data shape after handling missing values: (84, 18249)
For the feature 'Bile_Duct_Cancer', the least common label is '1.0' with 2 occurrences. This represents 2.38% of the dataset.
The distribution of the feature 'Bile_Duct_Cancer' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 35 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Bile_Duct_Cancer/cohort_info.json
The dataset was determined to be not usable for analysis due to bias in the trait distribution.
